In [1]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

/home/hlee053/miniconda3/envs/psb6351_environment/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  


In [10]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/home/hlee053/Documents/'
proj_dir = '/mattfeld_2020/sourcedata'
#subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/6-fMRI_DistortionMap_PA/resources/DICOM/files'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files'
#subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/9-fMRI_REVL_ROI_loc_2/resources/DICOM/files'
#subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [11]:
data_files

['/home/hlee053/Documents//mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-5-66-1m5xa6e.dcm',
 '/home/hlee053/Documents//mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-5-158-5mda29.dcm',
 '/home/hlee053/Documents//mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-5-24-1saj215.dcm',
 '/home/hlee053/Documents//mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-5-26-1njd4wu.dcm',
 '/home/hlee053/Documents//mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/5-T1w_MPR_vNav/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-5-101-1k1

In [12]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files[0])

In [13]:
data_set

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915240970098713566
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002, 0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM']
(0008, 0012) Instance Creation Date              DA: '20190619'
(0008, 0013) Instance Creation Time              TM: '152413.493000'
(0008, 0016) SOP Clas

In [14]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len(data_files)))
print('The timing of the TR was: {0} ms'.format(data_set.RepetitionTime))
print('The timing of the TE was: {0} ms'.format(data_set.EchoTime))
print('The flip angle in degrees was: {0}'.format(data_set.FlipAngle))
print('The name of the scan was: {0}'.format(data_set[int('00181030', 16)].value))
print('The phase encode direction was: {0}'.format(data_set[int('00181312', 16)].value))
print('COL = AP or PA; ROW = RL or LR')
#print('The slice timing was: {0}'.format(data_set[0x00191029].value))
print('The field of view - FOV was: {0}'.format(data_set[int('0051100c', 16)].value))

# Why use the following format to access information in the header?
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The phase encode direction was: {0}'.format(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]))
print('1 = +; 0 = -')
print('COL, 1 = PA; COL, 0 = AP')



The number of TRs collected equals: 176
The timing of the TR was: 2500 ms
The timing of the TE was: 2.9 ms
The flip angle in degrees was: 8
The name of the scan was: T1w_MPR_vNav
The phase encode direction was: ROW
COL = AP or PA; ROW = RL or LR
The field of view - FOV was: FoV 256*256
The phase encode direction was: 1
1 = +; 0 = -
COL, 1 = PA; COL, 0 = AP


# Using heudiconv to convert dicoms into BIDs format

In [7]:
# Step 1 - run heudiconv
# HeuDiConv(Heuristic Dicom Conversion) provides sophisticated and flexible creation of BIDS datasets. 
# It calls dcm2niix to convert the DICOMS to NIFTI with sidecar JSON files. 

# Install heudiconv through conda
# conda install -c conda-forge heudiconv

# Creating dset
# 
heudiconv -d '../sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/*' -s 021 -c none -f convertall -o /home/hlee053/Documents/mattfeld_2020/dset


# /Users/hlee/Documents/GitHub/mattfeld_2020/dset/.heudiconv/021/info


SyntaxError: invalid syntax (<ipython-input-7-afbabedbefa5>, line 10)

In [8]:
# os.path.join(path1, path2, ...): It combines the specified paths in order and makes them into one path.
heudiconvinfo_file = os.path.join(base_dir, 'mattfeld_2020', 'dset', '.heudiconv/021/info') #dset, .heudiconv/021/info -> hidden directory(can be found tying ls -a)

# pd.read_csv(): read csv file
dicominfo_df = pd.read_csv(heudiconvinfo_file + '/dicominfo.tsv', sep="\t")
limited_dicominfo_df = dicominfo_df[['series_id', 'series_files', 'dim1', 'dim2', 'dim3', 'dim4', 'series_description']]
# dim3 = slice_num
# dim4 = timepoints
limited_dicominfo_df

,series_id,series_files,dim1,dim2,dim3,dim4,series_description
0,1-localizer_32ch,3,512,512,3,1,localizer_32ch
1,2-T1w_MPR_vNav_setter,32,32,32,32,1,T1w_MPR_vNav_setter
2,3-T1w_MPR_vNav,145,32,32,32,145,T1w_MPR_vNav_setter
3,4-T1w_MPR_vNav,176,256,256,176,1,T1w_MPR_vNav
4,5-T1w_MPR_vNav,176,256,256,176,1,T1w_MPR_vNav
5,6-fMRI_DistortionMap_PA,66,100,100,66,1,fMRI_DistortionMap_PA
6,7-fMRI_DistortionMap_AP,66,100,100,66,1,fMRI_DistortionMap_AP
7,8-fMRI_REVL_ROI_loc_1,304,100,100,66,304,fMRI_REVL_ROI_loc_1
8,9-fMRI_REVL_ROI_loc_2,304,100,100,66,304,fMRI_REVL_ROI_loc_2
9,10-fMRI_REVL_Study_1,355,100,100,66,355,fMRI_REVL_Study_1


In [9]:
# After running psb6351_heuristic.py...?
# heudiconv -d "/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1////D//" -s 021 -f psb6351_heuristic.py -b -o /Users/amattfel/Documents/GitHub/mattfeld_2020/dset
heudiconv -d "/home/hlee053/Documents/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1////D//" -s 021 -f psb6351_heuristic.py -b -o /home/hlee053/Documents/mattfeld_2020/dset



SyntaxError: invalid syntax (<ipython-input-9-3d0fdcbdbfe5>, line 3)

In [12]:
dicominfo_df.keys()

NameError: name 'dicominfo_df' is not defined